In [ ]:
import sys
sys.path.insert(1, r"C:/Users/micha/Documents/Imperial Courses/Thesis/IAEA-thesis")
from Stage1.ExtractingGraphs.verifyGraphSize import verify_A_size, import_npz
from Stage1.ExtractingGraphs.HTMLtoGraph import EdgeFeatures
from Stage1.GAT.GAT import *